Handle Evolving Workflows
=========================

For some workflows we don't know the extent of the computation at the outset.  We need to do some computation in order to figure out the rest of the computation that we need to do.  The computation grows and evolves as we do more work.

As an example, consider a situation where you need to read many files and then based on the contents of those files, fire off additional work.  You would like to read the files in parallel, and then within each file expose more parallelism.

This example goes through three ways to handle this situation using [Dask Futures](https://docs.dask.org/en/latest/futures.html)

1.  Using `as_completed`
2.  Using `async/await`
3.  Launching tasks from tasks

But first, lets run our code sequentially.

0: Sequential code
------------------

In [1]:
filenames = ["file.{}.txt".format(i) for i in range(10)]

filenames[:3]

['file.0.txt', 'file.1.txt', 'file.2.txt']

In [2]:
import random, time


def parse_file(fn: str) -> list:
    """ Returns a list work items of unknown length """
    time.sleep(random.random())
    return [random.random() for _ in range(random.randint(1, 10))]

def process_item(x: float):
    """ Process each work item """
    time.sleep(random.random() / 4)
    return x + 1

In [3]:
%%time

# This takes around 10-20s

results = []

for fn in filenames:
    L = parse_file(fn)
    for x in L:
        out = process_item(x)
        results.append(out)

CPU times: user 1.69 ms, sys: 3.72 ms, total: 5.42 ms
Wall time: 11.5 s


Start Dask Client
-----------------

We'll need a Dask client in order to manage dynamic workloads

In [4]:
from dask.distributed import Client

client = Client(processes=False, n_workers=1, threads_per_worker=6)
client

Client Scheduler: inproc://10.20.1.11/10375/1 Dashboard: http://10.20.1.11:8787/status,Cluster Workers: 1 Cores: 6 Memory: 8.36 GB


1: Use as_completed
-------------------

The [as_completed](https://docs.dask.org/en/latest/futures.html#distributed.as_completed) iterator lets us handle futures as they complete.  We can then submit more data on the fly.

-   We submit a task for each of our filenames
-   We also compute the length of each of the returned lists
-   As those lengths return, we submit off a new task to get each item of that list.  We do this at higher priority, so that we process existing data before we collect new data.
-   We wait on all of the returned results

In [5]:
%%time

from dask.distributed import as_completed
import operator

lists = client.map(parse_file, filenames, pure=False)
lengths = client.map(len, lists)

mapping = dict(zip(lengths, lists))

futures = []

for future in as_completed(lengths):
    n = future.result()
    L = mapping[future]
    for i in range(n):
        new = client.submit(operator.getitem, L, i, priority=1)
        new = client.submit(process_item, new, priority=1)
        futures.append(new)
        
client.gather(futures)

CPU times: user 375 ms, sys: 48.1 ms, total: 423 ms
Wall time: 2.25 s


[1.8895608430915793,
 1.8718964930591095,
 1.2513834585680021,
 1.0986928964697869,
 1.3356704959236771,
 1.9847062857006674,
 1.9982877193839825,
 1.724792745499932,
 1.7300893506458976,
 1.2522497012980298,
 1.5124646890076563,
 1.9175521867544263,
 1.6617999681784306,
 1.8619568622843512,
 1.6394913246097584,
 1.1616159560043464,
 1.7266266639819356,
 1.211288738866802,
 1.6545743879044765,
 1.562768833290203,
 1.3064048009528568,
 1.714736845340097,
 1.597916865727565,
 1.3684865649808562,
 1.6881809255040894,
 1.668917964221046,
 1.7941051328279138,
 1.8346007049843176,
 1.4302759847677606,
 1.7651787013198552,
 1.8731135247302713,
 1.7577118768586575,
 1.7563994404944936,
 1.7689301071139194,
 1.422650622441037,
 1.9046521138584562,
 1.524307177353505,
 1.5444295562558625,
 1.469783211756034,
 1.1993238161151776,
 1.229286433522797,
 1.8797939812818916,
 1.1470502155678992,
 1.5522155102007016,
 1.8903059189354374,
 1.4938439244330854,
 1.960851002779081,
 1.4738400400901486,
 1.

2: Use async/await to handle single file processing locally
-----------------------------------------------------------

We can also handle the concurrency here within our local process.  This requires you to understand async/await syntax, but is generally powerful and arguably simpler than the `as_completed` approach above.

In [6]:
import asyncio

async def f(fn):
    """ Handle the lifecycle of a single file """
    future = client.submit(parse_file, fn, pure=False)
    length_future = client.submit(len, future)
    length = await length_future
    
    futures = [client.submit(operator.getitem, future, i, priority=10) 
               for i in range(length)]
    futures = client.map(process_item, futures, priority=10)
    return futures

async def run_all(filenames):
    list_of_list_of_futures = await asyncio.gather(*[f(fn) for fn in filenames])
    futures = sum(list_of_list_of_futures, [])
    return await client.gather(futures)


We now need to run this function in the same event loop as our client is running.  If we had started our client asynchronously, then we could have done this:

```python
client = await Client(asynchronous=True)

await run_all(filenames)
```

However, because we started our client without the `asynchronous=True` flag the event loop is actually running in a separate thread, so we'll have to ask the client to run this for us.

In [7]:
client.sync(run_all, filenames)

[1.5591021379039738,
 1.6665199766015313,
 1.5970921054509915,
 1.2495703451819982,
 1.5465696137629314,
 1.2358801423689387,
 1.05641013853539,
 1.5623502560653932,
 1.583479204397293,
 1.0305677111218912,
 1.9015820207364427,
 1.8726268811678932,
 1.2559606115124202,
 1.7258727244055247,
 1.1841800154627111,
 1.352719454078227,
 1.9336013718123473,
 1.273390708507515,
 1.112650731372271,
 1.6042264627511478,
 1.713924708288122,
 1.3037786001511082,
 1.0177959204702285,
 1.6881452071102094,
 1.2358322999876838,
 1.0576618198905718,
 1.391229063050717,
 1.507919484837919,
 1.4235425803002708,
 1.8367504658134857,
 1.2550041342138556,
 1.6620981904415815,
 1.3567782319347725,
 1.5086186675043893,
 1.5352162224025518,
 1.1442035170885294,
 1.9850851964421519,
 1.6447192364037038,
 1.0933914565794276,
 1.7962001503858105,
 1.783985257411862,
 1.6319152827157761,
 1.7600222731306527,
 1.0236957703499527,
 1.5302165035145765,
 1.9692941021320989,
 1.6405969912366933,
 1.876948746806817,
 1.

3: Submit tasks from tasks
--------------------------

We can also submit tasks that themselves submit more tasks.  See [documentation here](https://docs.dask.org/en/latest/futures.html#submit-tasks-from-tasks).

In [8]:
%%time

from dask.distributed import get_client, secede, rejoin

def f(fn):
    L = parse_file(fn)
    client = get_client()
    
    futures = client.map(process_item, L, priority=10)
    secede()
    results = client.gather(futures)
    rejoin()
    return results

futures = client.map(f, filenames, pure=False)
results = client.gather(futures)

CPU times: user 259 ms, sys: 3.83 ms, total: 263 ms
Wall time: 2 s
